참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import matplotlib.patches as patch # 도형 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import sympy.plotting as splot
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

## 08.005 평면 응력의 변환<br>08.005 2D Stress Transform

2차원 응력 상태를 바라보는 각도에 따라 좌표 변환<br>Coordinate transformation of 2D stress status

p. 346

### 2차원 응력 상태<br>2D stress status

$$
M=\begin{bmatrix}
\sigma_x & \tau_{xy} \\
\tau_{xy} & \sigma_y \\
\end{bmatrix}
$$

In [ ]:
sigma_x_Pa, sigma_y_Pa, tau_xy_Pa = sy.symbols('sigma_{x}[Pa], sigma_{y}[Pa], tau_{xy}[Pa]', real=True)

In [ ]:
mat_M = sy.Matrix([(sigma_x_Pa, tau_xy_Pa), 
                   (tau_xy_Pa, sigma_y_Pa)])

In [ ]:
mat_M

아래 값은 예제 8.4로 부터 가져온 것임<br>Following values are from example 8.4

In [ ]:
s_d = {
    sigma_x_Pa: 30e6,
    sigma_y_Pa: 60e6,
    tau_xy_Pa: 40e6,
}

응력 상태 표시 준비<br>Prepare for stress status plot

In [ ]:
import os   # 운영체제 관련 기능 Operating Systems
import sys  # 시스템 관련 기능 Systems
# utils 폴더의 모듈을 import 할 수 있도록 준비
# add utils folder to sys.path to import
sys.path.append(os.path.abspath(os.path.join(os.pardir, 'utils')))
# 선도 관련 기능 diagrams
import draw_diagrams

응력상태를 표시함<br>Draw stress status

In [ ]:
draw_diagrams.draw_stress_2d(sigma_x_Pa.subs(s_d), 
                             sigma_y_Pa.subs(s_d), 
                             tau_xy_Pa.subs(s_d))
plt.show()

응력을 바라보는 각도가 달라지면 응력 상태가 다르게 관찰됨<br>If an engineer changes the angle of observation, the engineer can observe the stress differently.

In [ ]:
s_d_30_deg = {
    sigma_x_Pa: 72.1e6,
    sigma_y_Pa: 17.9e6,
    tau_xy_Pa: 33e6,
}

아래의 두 응력 상태는 단지 관찰되는 각도만 다른 것임<br>Following two stress status are the same but only different in which angle the engineer is observing

In [ ]:
ax0 = plt.subplot(1, 2, 1)
draw_diagrams.draw_stress_2d(sigma_x_Pa.subs(s_d), 
                             sigma_y_Pa.subs(s_d), 
                             tau_xy_Pa.subs(s_d), ax=ax0)
ax30 = plt.subplot(1, 2, 2)
draw_diagrams.draw_stress_2d(sigma_x_Pa.subs(s_d_30_deg), 
                             sigma_y_Pa.subs(s_d_30_deg), 
                             tau_xy_Pa.subs(s_d_30_deg),
                             ax=ax30,
                             angle_deg=30)
plt.show()

### 힘의 평형<br>Equilibrium of forces

그림 8.10 c)에서 빗면의 면적이 $dA$ 라면 밑면의 면적 $A_b$ 와 옆면의 면적 $A_f$ 는 다음과 같음<br>From fig. 8.10 c), if the area of the hypotenuse is $dA$, the areas of base $A_b$ and face $A_f$ are as follows.

$$
A_b = dA sin \theta \\
A_f = dA cos \theta
$$

그림 8.10 c)와 d) 에서 힘 평형 방정식은 다음과 같음<br>Equations of equilibrum for figures 8.10 c) and d) are as follows.

$$
\begin{cases}
\sigma_{x'}dAcos\theta = \tau_{x'y'} dA sin\theta + \sigma_x A_f + \tau_{xy} A_b \\
\sigma_{x'}dAsin\theta + \tau_{x'y'} dA cos\theta = \tau_{xy} A_f + \sigma_y A_b\\
\end{cases}
$$

$$
\begin{cases}
\sigma_{x'}dAcos\theta - \tau_{x'y'} dA sin\theta = \sigma_x dAcos\theta + \tau_{xy}  dAsin\theta \\
\sigma_{x'}dAsin\theta + \tau_{x'y'} dA cos\theta = \tau_{xy} dAcos\theta + \sigma_y dAsin\theta \\
\end{cases}
$$

양변을 $dA$로 나눔<br>Divide both sides by $dA$

$$
\begin{cases}
\sigma_{x'}cos\theta - \tau_{x'y'} sin\theta = \sigma_x cos\theta + \tau_{xy}  sin\theta \\
\sigma_{x'}sin\theta + \tau_{x'y'} cos\theta = \tau_{xy} cos\theta + \sigma_y sin\theta \\
\end{cases}
$$

#### 행렬 형태로 <br>In matrix form

$$
    \begin{bmatrix}
        cos\theta & -sin\theta \\
        sin\theta & cos\theta\\
    \end{bmatrix}
    \begin{pmatrix}
        \sigma_{x'}\\
        \tau_{x'y'}\\
    \end{pmatrix}=
    \begin{bmatrix}
        \sigma_{x} & \tau_{xy} \\
        \tau_{xy} & \sigma_{y}\\
    \end{bmatrix}
    \begin{pmatrix}
        cos\theta\\
        sin\theta\\
    \end{pmatrix}
$$

회전 행렬의 역행렬<br>Inverse of the rotation matrix

$$
    \begin{bmatrix}
        cos\theta & -sin\theta \\
        sin\theta & cos\theta\\
    \end{bmatrix} ^ {-1}
=
    \begin{bmatrix}
        cos(-\theta) & -sin(-\theta) \\
        sin(-\theta) & cos(-\theta)\\
    \end{bmatrix}
=
    \begin{bmatrix}
        cos\theta & sin\theta \\
        -sin\theta & cos\theta\\
    \end{bmatrix}
$$

회전행렬의 역행렬을 곱함<br>Multiply inverse of the rotation matrix

$$
\begin{align}
    \begin{bmatrix}
        cos\theta & sin\theta \\
        -sin\theta & cos\theta\\
    \end{bmatrix}
    \begin{bmatrix}
        cos\theta & -sin\theta \\
        sin\theta & cos\theta\\
    \end{bmatrix}
& =
    \begin{bmatrix}
        cos^2\theta + sin^2\theta & - cos\theta sin\theta + sin\theta cos\theta \\
        -sin\theta cos\theta + cos\theta sin\theta &  sin^2\theta + cos^2\theta \\
    \end{bmatrix} \\
& =
    \begin{bmatrix}
        1 & 0 \\
        0 & 1 \\
    \end{bmatrix} \\
\end{align}
$$

따라서<br>Thus

$$
    \begin{pmatrix}
        \sigma_{x'}\\
        \tau_{x'y'}\\
    \end{pmatrix}=
    \begin{bmatrix}
        cos\theta & sin\theta \\
        -sin\theta & cos\theta\\
    \end{bmatrix}
    \begin{bmatrix}
        \sigma_{x} & \tau_{xy} \\
        \tau_{xy} & \sigma_{y}\\
    \end{bmatrix}
    \begin{pmatrix}
        cos\theta\\
        sin\theta\\
    \end{pmatrix}
$$

한편 $\sigma_{y'}$ 는 다음과 같이 구할 수 있다.

$$
\begin{align}
\sigma_{y'} & =
    \begin{pmatrix}
        cos\left(\theta+\frac{\pi}{2}\right) & sin\left(\theta+\frac{\pi}{2}\right)
    \end{pmatrix}   
    M
    \begin{pmatrix}
        cos\left(\theta+\frac{\pi}{2}\right) \\
        sin\left(\theta+\frac{\pi}{2}\right) \\
    \end{pmatrix}
\end{align}
$$

$$
\begin{align}
\sigma_{y'} & =
    \begin{pmatrix}
        -sin\theta & cos\theta
    \end{pmatrix}   
    M
    \begin{pmatrix}
        -sin\theta \\
        cos\theta \\
    \end{pmatrix}
\end{align}
$$

다시 한번 행렬 형태로 정리하면<br>Again in matrix form

$$
    \begin{bmatrix}
        \sigma_{x'} & \tau_{x'y'} \\
        \tau_{x'y'} & \sigma_{y'} \\
    \end{bmatrix}=
    \begin{bmatrix}
        cos\theta & sin\theta \\
        -sin\theta & cos\theta\\
    \end{bmatrix}
    \begin{bmatrix}
        \sigma_{x} & \tau_{xy} \\
        \tau_{xy} & \sigma_{y}\\
    \end{bmatrix}
    \begin{bmatrix}
        cos\theta & -sin\theta \\
        sin\theta & cos\theta \\
    \end{bmatrix}
$$

#### `sympy`를 이용하여<br>Using `sympy`

사용할 기호를 준비<br>Prepare symbols

In [ ]:
sigma_xp_Pa, sigma_yp_Pa, tau_xpyp_Pa = sy.symbols('sigma_{xp}[Pa], sigma_{yp}[Pa], tau_{xpyp}[Pa]', real=True)

In [ ]:
Mp = sy.Matrix([(sigma_xp_Pa, tau_xpyp_Pa), (tau_xpyp_Pa, sigma_yp_Pa)])

In [ ]:
Mp

$$
\begin{cases}
\sigma_{x'}cos\theta - \tau_{x'y'} sin\theta = \sigma_x cos\theta + \tau_{xy}  sin\theta \\
\sigma_{x'}sin\theta + \tau_{x'y'} cos\theta = \tau_{xy} cos\theta + \sigma_y sin\theta \\
\end{cases}
$$

In [ ]:
theta_rad = sy.symbols('theta[rad]', real=True)

x 방향 평형<br>x direction equilibrium

In [ ]:
x_eq = sy.Eq(sigma_xp_Pa * sy.cos(theta_rad) - tau_xpyp_Pa * sy.sin(theta_rad),
             sigma_x_Pa * sy.cos(theta_rad) + tau_xy_Pa * sy.sin(theta_rad)
            )

In [ ]:
x_eq

y 방향 평형<br>y direction equilibrium

In [ ]:
y_eq = sy.Eq(sigma_xp_Pa * sy.sin(theta_rad) + tau_xpyp_Pa * sy.cos(theta_rad),
             tau_xy_Pa * sy.cos(theta_rad) + sigma_y_Pa * sy.sin(theta_rad)
            )

In [ ]:
y_eq

$\sigma_{x'}$ 와 $\tau_{x'y'}$에 대해 풂<br>Solve for $\sigma_{x'}$ & $\tau_{x'y'}$

In [ ]:
stress_sol = sy.solve((x_eq, y_eq), (sigma_xp_Pa, tau_xpyp_Pa))

In [ ]:
stress_sol

In [ ]:
sigma_xp_sol_Pa = stress_sol[sigma_xp_Pa].subs(
    {
        sy.sin(theta_rad) ** 2: (1 - sy.cos(2 * theta_rad)) / 2,
        sy.cos(theta_rad) ** 2: (1 + sy.cos(2 * theta_rad)) / 2,
    }
)

In [ ]:
sigma_xp_sol_Pa

In [ ]:
tau_xpyp_sol_Pa = stress_sol[tau_xpyp_Pa]

In [ ]:
tau_xpyp_sol_Pa

$\sigma_{y'}$ = $\left[ \sigma_{x'} \right]_{\theta+\frac{1}{2}\pi}$

In [ ]:
sigma_yp_Pa = sigma_xp_sol_Pa.subs(theta_rad, theta_rad + sy.pi / 2)

In [ ]:
sigma_yp_Pa

$\sigma_{x'} + \sigma_{y'}$

In [ ]:
sy.simplify(sigma_xp_sol_Pa + sigma_yp_Pa)

### 주응력 방향과 주응력<br>Principal directions and Principal stresses

수직 응력 $\sigma_{x'}$가 최대가 되는 $\theta$를 구해보자.<br>Let's find $\theta$ maximizing $\sigma_{x'}$.

$\sigma_{x'}$ 를 $\theta$로 미분<br>Differentiate $\sigma_{x'}$ with $\theta$

In [ ]:
dsigma_dt = sy.simplify(sy.diff(sigma_xp_sol_Pa, theta_rad))

In [ ]:
dsigma_dt

$\tau_{x'y'}$ 와 이 결과를 비교해 보자.<br>Let's compare this result with $\tau_{x'y'}$.

In [ ]:
sy.simplify(dsigma_dt / tau_xpyp_sol_Pa)

이 결과의 의미는 무엇인가?<br>What does this result mean?

$\frac{d\sigma_{x'}}{d\theta}$ 를 0 으로 만드는 $\theta$를 구한다.<br>Find $\theta$ making $\frac{d\sigma_{x'}}{d\theta}$  zero.

In [ ]:
theta_sigma_sol = sy.solve(dsigma_dt, theta_rad)

In [ ]:
theta_sigma_sol

제곱근 안을 좀 더 간단하게<br>To make the inside of the square root simpler

In [ ]:
theta_sigma = theta_sigma_sol[0].subs(sy.expand((sigma_x_Pa - sigma_y_Pa) ** 2), (sigma_x_Pa - sigma_y_Pa) ** 2)

In [ ]:
theta_sigma

위 대입의 결과가 원래 결과와 같은지 확인<br>Assert if result above is the same as the original result

In [ ]:
assert (0 == sy.simplify(theta_sigma - theta_sigma_sol[0]))

구해진 $\theta$를 $\sigma_{x'}$, $\sigma_{y'}$, $\tau_{x'y'}$에 대입 (해당 결과는 다소 복잡해 보일 수 있음)<br>Substitute $\theta$ to $\sigma_{x'}$, $\sigma_{y'}$, $\tau_{x'y'}$ (The results may seem more complicated than desired)

In [ ]:
s1 = sy.simplify(sigma_xp_sol_Pa.subs(theta_rad, theta_sigma)) 

In [ ]:
s2 = sy.simplify(sigma_yp_Pa.subs(theta_rad, theta_sigma)) 

In [ ]:
t1 = sy.simplify(tau_xpyp_sol_Pa.subs(theta_rad, theta_sigma)) 

In [ ]:
t1

예제 8.4 의 값을 대입하면 다음과 같다.<br>Using values from ex 8.4

In [ ]:
sigma_1_Pa = float(sy.simplify(s1.subs(s_d)))

In [ ]:
sigma_1_Pa

In [ ]:
sigma_2_Pa = float(sy.simplify(s2.subs(s_d)))

In [ ]:
sigma_2_Pa

In [ ]:
theta_sigma_max_deg = np.rad2deg(float(theta_sigma.subs(s_d)))

In [ ]:
theta_sigma_max_deg

이미 알고 있던 값과 확인<br>Assert using known values

In [ ]:
assert(1e-6 > abs(tau_xpyp_sol_Pa.subs(theta_rad, theta_sigma).subs(s_d)))

In [ ]:
assert(1e-6 > abs(sigma_1_Pa - 87720018.72658765)/87720018.72658765)

In [ ]:
assert(1e-6 > abs(sigma_2_Pa - 2279981.2734123357)/2279981.2734123357)

In [ ]:
assert(1e-6 > abs(theta_sigma_max_deg - 55.2780226098)/55.2780226098)

응력 상태를 그림으로 표시<br>Plot stress status

In [ ]:
ax0 = plt.subplot(1, 2, 1)
draw_diagrams.draw_stress_2d(sigma_x_Pa.subs(s_d), 
                             sigma_y_Pa.subs(s_d), 
                             tau_xy_Pa.subs(s_d), ax=ax0)
axmax = plt.subplot(1, 2, 2)
draw_diagrams.draw_stress_2d(sigma_1_Pa, 
                             sigma_2_Pa, 
                             0,
                             ax=axmax,
                             angle_deg=theta_sigma_max_deg)
plt.show()

### 최대 면내 전단 응력<br>Maximum in-plane shear stress

전단 응력 $\tau_{x'y'}$가 최대가 되는 $\theta$를 구해보자.<br>Let's find $\theta$ maximizing $\tau_{x'y'}$.

$\tau_{x'y'}$ 를 $\theta$로 미분<br>Differentiate $\tau_{x'y'}$ with $\theta$

In [ ]:
dtau_dt = sy.simplify(sy.diff(tau_xpyp_sol_Pa, theta_rad))

In [ ]:
dtau_dt

$\frac{d\tau_{x'y'}}{d\theta}$ 를 0 으로 만드는 $\theta$를 구한다.<br>Find $\theta$ making $\frac{d\tau_{x'y'}}{d\theta}$  zero.

In [ ]:
theta_tau_sol = sy.solve(dtau_dt, theta_rad)

In [ ]:
theta_tau_sol

In [ ]:
theta_tau = theta_tau_sol[0]

구해진 $\theta$를 $\sigma_{x'}$, $\sigma_{y'}$, $\tau_{x'y'}$에 대입<br>Substitute $\theta$ to $\sigma_{x'}$, $\sigma_{y'}$, $\tau_{x'y'}$

In [ ]:
st1 = sy.trigsimp(sigma_xp_sol_Pa.subs(theta_rad, theta_tau)) 

In [ ]:
st2 = sy.trigsimp(sigma_yp_Pa.subs(theta_rad, theta_tau)) 

In [ ]:
t0 = sy.trigsimp(tau_xpyp_sol_Pa.subs(theta_rad, theta_tau)) 

예제 8.4 의 값을 대입하면 다음과 같다.

In [ ]:
sigma_tau_1_Pa = float(sy.simplify(st1.subs(s_d)))

In [ ]:
sigma_tau_1_Pa

In [ ]:
sigma_tau_2_Pa = float(sy.simplify(st2.subs(s_d)))

In [ ]:
sigma_tau_2_Pa

In [ ]:
tau_1_Pa = float(sy.simplify(t0.subs(s_d)))

In [ ]:
tau_1_Pa

In [ ]:
theta_tau_max_deg = np.rad2deg(float(theta_tau.subs(s_d)))

In [ ]:
theta_tau_max_deg

이미 알고 있던 값과 확인<br>Assert using known values

In [ ]:
assert(1e-6 > abs(sigma_tau_1_Pa - 45000000.0)/45000000.0)

In [ ]:
assert(1e-6 > abs(sigma_tau_2_Pa - 45000000.0)/45000000.0)

In [ ]:
assert(1e-6 > abs(tau_1_Pa - 42720018.72658765)/42720018.72658765)

In [ ]:
assert(1e-6 > abs(theta_tau_max_deg - 10.2780226098)/10.2780226098)

응력 상태를 그림으로 표시<br>Plot stress status

In [ ]:
ax0 = plt.subplot(1, 2, 1)
draw_diagrams.draw_stress_2d(sigma_x_Pa.subs(s_d), 
                             sigma_y_Pa.subs(s_d), 
                             tau_xy_Pa.subs(s_d), ax=ax0)
axmax = plt.subplot(1, 2, 2)
draw_diagrams.draw_stress_2d(sigma_tau_1_Pa, 
                             sigma_tau_2_Pa, 
                             tau_1_Pa,
                             ax=axmax,
                             angle_deg=theta_tau_max_deg)
plt.show()